In [ ]:
import dcarte
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec

import pyarrow as pa
import pyarrow.parquet as pq

import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
maps= pd.read_excel('dyads_unprotected.xlsx')
maps['id'] = maps['Research ID'].apply(lambda x:x[:5])

sorted_ids = pd.read_csv('sorted_ids_14_11_2023.csv')
ids = sorted_ids['participant_id'].unique()
maps = maps[maps['id'].isin(ids)][['id','DYAD']]
len(maps)

In [ ]:
npi = pd.read_csv('redcap_16_nov_2023/NPI_MinderHealthManageme_DATA_LABELS_2023-11-16_1120.csv')
#npi.columns.to_list()


In [ ]:
npi = npi.rename(columns={'Participants ID':'DYAD'})
print(len(npi))
npi = npi.merge(maps[['id','DYAD']], on='DYAD',how='inner')
print(len(npi))
print(len(npi['id'].unique()))
#1294,455,67

In [ ]:
for i in ['','.1','.2','.3','.4','.5','.6','.7','.8','.9','.10','.11']:
    list1, list2 = npi['Severity'+i].unique(),npi['Frequency'+i].unique()
    a = [x.split(' - ')[0] if type(x)==str else 0 for x in list1]
    b =[x.split(' - ')[0] if type(x)==str else 0 for x in list2]
    print(a, b)

In [ ]:
npi = npi[['id','Event Name','Assessment date and time', 'Data source',
           'Frequency','Severity','Frequency.1','Severity.1','Frequency.2','Severity.2','Frequency.3','Severity.3',
           'Frequency.4','Severity.4','Frequency.5','Severity.5','Frequency.6','Severity.6','Frequency.7','Severity.7',
           'Frequency.8','Severity.8','Frequency.9','Severity.9','Frequency.10','Severity.10','Frequency.11','Severity.11',
           'Total Frequency', 'Total Severity','FrequencyXSeverity', 'Complete?','Total Distress Score','SLEEP:  Does the patient have difficulty sleeping (do not count as present if the patient simply gets up once or twice per night only to go to the bathroom and falls back asleep immediately)? Is he/she up at night? Does he/she wander at night, get dressed, or disturb your sleep? ',
         'Tick all that apply (choice=Does the patient have difficulty falling asleep?)',
         'Tick all that apply (choice=Does the patient get up during the night (do not count if the patient gets up once or twice per night only to go to the bathroom and falls back asleep immediately)?)',
         'Tick all that apply (choice=Does the patient wander, pace, or get involved in inappropriate activities at night?)',
         'Tick all that apply (choice=Does the patient awaken you during the night?)',
         'Tick all that apply (choice=Does the patient awaken at night, dress, and plan to go out thinking that it is morning and time to start the day?)',
         'Tick all that apply (choice=Does the patient awaken too early in the morning (earlier that was his/her habit)?)',
         'Tick all that apply (choice=Does the patient sleep excessively during the day?)',
         'Tick all that apply (choice=Does the patient have any other nighttime behaviours that bother you that we havent talked about?)']]
npi = npi[npi['Complete?']=='Complete'].reset_index(drop=True)

categories = ['delusions','hallucinations','agitation_aggression','depression_dysphoria',
             'anxiety','elation_euphoria','apathy','disinhibition',
             'irritability','aberrant_motor_beh','sleep','appetite']

new_names = {'Frequency':'freq_delusions','Severity':'sev_delusions',
             'Frequency.1':'freq_hallucinations','Severity.1':'sev_hallucinations',
             'Frequency.2':'freq_agitation_aggression','Severity.2':'sev_agitation_aggression',
             'Frequency.3':'freq_depression_dysphoria','Severity.3':'sev_depression_dysphoria',
             'Frequency.4':'freq_anxiety','Severity.4':'sev_anxiety',
             'Frequency.5':'freq_elation_euphoria','Severity.5':'sev_elation_euphoria',
             'Frequency.6':'freq_apathy','Severity.6':'sev_apathy',
             'Frequency.7':'freq_disinhibition','Severity.7':'sev_disinhibition',
             'Frequency.8':'freq_irritability','Severity.8':'sev_irritability',
             'Frequency.9':'freq_aberrant_motor_beh','Severity.9':'sev_aberrant_motor_beh',
             'Frequency.10':'freq_sleep','Severity.10':'sev_sleep',
             'Frequency.11':'freq_appetite','Severity.11':'sev_appetite'}

freq_dict ={'Very frequently':4, 'Often':2, 'Frequently':3, 'Occasionally':1}
sev_dict = {'Mild':1, 'Marked':3, 'Moderate':2}

for idx,i in enumerate(npi.columns[4:28]):    
    npi[i]=npi[i].apply(lambda x: x.split(' - ')[0] if type(x)==str else x)
    npi = npi.rename(columns={i:new_names[i]})

for i in categories:
    npi['freq_'+i] = npi['freq_'+i].apply(lambda x:freq_dict[x] if type(x)==str else 0)
    npi['sev_'+i] = npi['sev_'+i].apply(lambda x:sev_dict[x] if type(x)==str else 0)
    npi[i] = npi['freq_'+i]*npi['sev_'+i]

for idx,i in enumerate(npi.columns[34:42]):
    new_label = i.split('=')
    npi[i]=npi[i].apply(lambda x: 'Yes' if x=='Checked' else 'No')
    npi = npi.rename(columns={i:new_label[1]})
    
npi.head(3)

In [ ]:
npi['thalias_total_frequency']= npi[['freq_delusions', 'freq_hallucinations','freq_agitation_aggression',
        'freq_depression_dysphoria','freq_anxiety', 'freq_elation_euphoria', 'freq_apathy',
       'freq_disinhibition', 'freq_irritability',  'freq_aberrant_motor_beh','freq_sleep',  'freq_appetite']].sum(axis=1)

In [ ]:
npi['thalias_total_sev']= npi[['sev_delusions', 'sev_hallucinations','sev_agitation_aggression',
        'sev_depression_dysphoria','sev_anxiety', 'sev_elation_euphoria', 'sev_apathy',
       'sev_disinhibition', 'sev_irritability',  'sev_aberrant_motor_beh','sev_sleep',  'sev_appetite']].sum(axis=1)

In [ ]:
npi['thalias_total'] = npi[categories].sum(axis=1)
npi[['thalias_total_frequency','Total Frequency','thalias_total_sev','Total Severity','FrequencyXSeverity','thalias_total']].head(6)    

In [ ]:
len(npi)

In [ ]:
len(npi[npi['thalias_total_frequency']==npi['Total Frequency']])

In [ ]:
len(npi[npi['thalias_total_sev']==npi['Total Severity']])

In [ ]:
npi['datetime'] = pd.to_datetime(npi['Assessment date and time'], dayfirst=True)

initial_dates = npi[['id','datetime']][npi['Event Name']=='Baseline']
initial_dates = initial_dates.rename(columns={'datetime':'baseline_date'}).reset_index()
npi = npi.merge(initial_dates, on=['id'],how='left')
npi['diff_with_baseline'] = (npi['datetime'] - npi['baseline_date']).dt.days
npi[['id', 'Event Name', 'Assessment date and time','datetime']]

In [ ]:
npi['is']='no exit'
npi.loc[(npi['Event Name']=='Exit'), 'is'] = 'exit'
npi.loc[(npi['Event Name']=='Unscheduled visit'), 'is'] = 'unschedule'
npi.loc[(npi['Event Name']=='Baseline'), 'event_numeric'] = 0
npi.loc[(npi['Event Name']=='Month 3'), 'event_numeric'] = 3
npi.loc[(npi['Event Name']=='Month 6'), 'event_numeric'] = 6
npi.loc[(npi['Event Name']=='Month 9'), 'event_numeric'] = 9
npi.loc[(npi['Event Name']=='Month 12'), 'event_numeric'] = 12
npi.loc[(npi['Event Name']=='Month 15'), 'event_numeric'] = 15
npi.loc[(npi['Event Name']=='Month 18'), 'event_numeric'] = 18
npi.loc[(npi['Event Name']=='Month 21'), 'event_numeric'] = 21
npi.loc[(npi['Event Name']=='Month 24'), 'event_numeric'] = 24
npi.loc[(npi['Event Name']=='Month 27'), 'event_numeric'] = 27
npi.loc[(npi['Event Name']=='Month 30'), 'event_numeric'] = 30
npi.loc[(npi['Event Name']=='Month 33'), 'event_numeric'] = 33
npi.loc[(npi['Event Name']=='Month 36'), 'event_numeric'] = 36
npi['event_numeric'] = npi.apply(lambda x: (x['diff_with_baseline']/30 if x['is']=='exit' else x['event_numeric']), axis=1)
npi['event_numeric'] = npi.apply(lambda x: (x['diff_with_baseline']/30 if x['is']=='unschedule' else x['event_numeric']), axis=1)

npi = npi[['id', 'Event Name','event_numeric', 'datetime','baseline_date','diff_with_baseline',
        'FrequencyXSeverity','thalias_total','thalias_total_frequency','Total Frequency','thalias_total_sev','Total Severity','Total Distress Score',
        'freq_sleep', 'sev_sleep', 'sleep', 'freq_delusions', 'sev_delusions', 'delusions','freq_hallucinations',
       'sev_hallucinations',  'hallucinations',  'freq_agitation_aggression',
       'sev_agitation_aggression','agitation_aggression', 'freq_depression_dysphoria',
       'sev_depression_dysphoria', 'depression_dysphoria','freq_anxiety', 'sev_anxiety',  'anxiety',
       'freq_elation_euphoria', 'sev_elation_euphoria','elation_euphoria', 'freq_apathy',
       'sev_apathy', 'apathy','freq_disinhibition', 'sev_disinhibition', 'disinhibition', 
       'freq_irritability', 'sev_irritability','irritability',  'freq_aberrant_motor_beh',
       'sev_aberrant_motor_beh','aberrant_motor_beh','freq_appetite',
       'sev_appetite', 'appetite','is','Data source','Complete?','SLEEP:  Does the patient have difficulty sleeping (do not count as present if the patient simply gets up once or twice per night only to go to the bathroom and falls back asleep immediately)? Is he/she up at night? Does he/she wander at night, get dressed, or disturb your sleep? ',
       'Does the patient have difficulty falling asleep?)',
       'Does the patient get up during the night (do not count if the patient gets up once or twice per night only to go to the bathroom and falls back asleep immediately)?)',
       'Does the patient wander, pace, or get involved in inappropriate activities at night?)',
       'Does the patient awaken you during the night?)',
       'Does the patient awaken at night, dress, and plan to go out thinking that it is morning and time to start the day?)',
       'Does the patient awaken too early in the morning (earlier that was his/her habit)?)',
       'Does the patient sleep excessively during the day?)',
       'Does the patient have any other nighttime behaviours that bother you that we havent talked about?)']]

In [ ]:
npi.to_excel('npi_values_30_NOV_23.xlsx')

In [ ]:
my_ids = npi.sort_values(by=['id'])['id'].unique()
my_ids

g = sns.FacetGrid(npi, col='id', col_order=my_ids, col_wrap=10, height=2)#, ylim=(0, 10))
g.map(sns.scatterplot, 'event_numeric', 'diff_with_baseline','is')
g.add_legend()

In [ ]:
br

In [ ]:
sns.scatterplot(data=npi,x='thalias_total_sev',y='Total Severity')

In [ ]:
sns.scatterplot(data=npi,x='thalias_total_frequency',y='Total Frequency')

In [ ]:
sns.scatterplot(data=npi,x='Total Distress Score',y='thalias_total')
br